<a href="https://colab.research.google.com/github/haider14-9abbaas/IoT-Security-System/blob/main/IoT%20Security%20System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install the cryptography library
!pip install cryptography


In [ ]:
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import serialization

# Generate RSA private key
private_key = rsa.generate_private_key(public_exponent=65537, key_size=2048)

# Serialize private key
private_key_pem = private_key.private_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.PrivateFormat.PKCS8,
    encryption_algorithm=serialization.NoEncryption()
)

# Generate public key
public_key = private_key.public_key()

# Serialize public key
public_key_pem = public_key.public_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.PublicFormat.SubjectPublicKeyInfo
)

# Save keys to files
with open("root_private_key.pem", "wb") as f:
    f.write(private_key_pem)

with open("root_public_key.pem", "wb") as f:
    f.write(public_key_pem)

print("Root CA private and public keys generated and saved.")


Root CA private and public keys generated and saved.


In [ ]:
import time

print("Simulating certificate issuance...")

# Example certificates
devices = ["light-sensor.smarthome.com", "user.smarthome.com"]
for device in devices:
    print(f"Generating certificate for {device}...")
    # Simulate certificate creation
    time.sleep(1)  # Simulate processing time
    print(f"Certificate for {device} generated and signed by Root CA.")



Simulating certificate issuance...
Generating certificate for light-sensor.smarthome.com...
Certificate for light-sensor.smarthome.com generated and signed by Root CA.
Generating certificate for user.smarthome.com...
Certificate for user.smarthome.com generated and signed by Root CA.


In [ ]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
import os

# Sensor data
plaintext = b"Sensor data: Temperature = 22.5C, Humidity = 45%."

# Generate AES key and IV
key = os.urandom(32)  # 256-bit key
iv = os.urandom(16)   # 128-bit IV

# Encrypt data
cipher = Cipher(algorithms.AES(key), modes.CFB(iv))
encryptor = cipher.encryptor()
ciphertext = encryptor.update(plaintext) + encryptor.finalize()

print("Original plaintext:", plaintext)
print("Ciphertext:", ciphertext)

# Decrypt data
decryptor = cipher.decryptor()
decrypted_text = decryptor.update(ciphertext) + decryptor.finalize()
print("Decrypted plaintext:", decrypted_text)


Original plaintext: b'Sensor data: Temperature = 22.5C, Humidity = 45%.'
Ciphertext: b'\x83\xccCO\xf5\x91\xda\xf1d!&\x1e\xd81\xb3>\xb3\xcdzZwR\x994\x12\xcd\xb7L\xc4\x95\x91I\xbb\x1d\x1d\xd7\x1d\n\x95\x954\xdd\xc8\x1d\x85\xebOH)'
Decrypted plaintext: b'Sensor data: Temperature = 22.5C, Humidity = 45%.'


In [ ]:
from cryptography.hazmat.primitives import hashes

# Create SHA-256 hash
digest = hashes.Hash(hashes.SHA256())
digest.update(plaintext)
data_hash = digest.finalize()

print("Data hash (SHA-256):", data_hash.hex())


Data hash (SHA-256): df75caedbf1fbe93eea2bd340b227a8dc16c6700d9b101a2ae94c4dbef3fb6e7


In [ ]:
import time

# Generate timestamp and nonce
timestamp = int(time.time())
nonce = os.urandom(16).hex()

print("Generated timestamp:", timestamp)
print("Generated nonce:", nonce)


Generated timestamp: 1736839289
Generated nonce: 5c66bcf4711c356e126f1692c609078f


In [ ]:
print("Simulating mutual authentication...")

# Example validation process
device_certificate_valid = True  # Simulating certificate validity check

if device_certificate_valid:
    print("Authentication successful: Device certificate is valid and signed by Root CA.")
else:
    print("Authentication failed: Invalid device certificate.")


Simulating mutual authentication...
Authentication successful: Device certificate is valid and signed by Root CA.


In [ ]:
import os
import time
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes

# Initialize devices and data
devices = ["temperature-sensor.smarthome.com", "light-sensor.smarthome.com", "cleaning-sensor.smarthome.com"]
sensor_data = {}

# Function to simulate certificate issuance
def issue_certificate(devices):
    print("\nSimulating certificate issuance for devices...")
    for device in devices:
        print(f"Generating certificate for {device}...")
        time.sleep(1)  # Simulate processing time
        print(f"Certificate for {device} generated and signed by Root CA.")

# Function to add a new device
def add_device():
    device_name = input("Enter the new device name (e.g., 'motion-sensor.smarthome.com'): ")
    if device_name not in devices:
        devices.append(device_name)
        print(f"Device '{device_name}' added successfully.")
    else:
        print("Device already exists!")

# Function to input sensor data
def input_sensor_data():
    for device in devices:
        data = input(f"Enter data for {device} (e.g., 'Temperature = 24C'): ").encode()
        sensor_data[device] = data
    print("Sensor data recorded.")

# Function to secure communication (AES encryption)
def secure_communication():
    print("\nSecuring communication for devices...")
    for device, data in sensor_data.items():
        print(f"\nEncrypting data for {device}...")
        key = os.urandom(32)  # 256-bit AES key
        iv = os.urandom(16)   # 128-bit IV

        # Encrypt data
        cipher = Cipher(algorithms.AES(key), modes.CFB(iv))
        encryptor = cipher.encryptor()
        ciphertext = encryptor.update(data) + encryptor.finalize()

        print(f"Ciphertext for {device}: {ciphertext}")

        # Decrypt data
        decryptor = cipher.decryptor()
        decrypted_text = decryptor.update(ciphertext) + decryptor.finalize()
        print(f"Decrypted plaintext for {device}: {decrypted_text}")

# Function to ensure data integrity
def ensure_data_integrity():
    print("\nEnsuring data integrity for devices...")
    for device, data in sensor_data.items():
        digest = hashes.Hash(hashes.SHA256())
        digest.update(data)
        data_hash = digest.finalize()
        print(f"Data hash for {device} (SHA-256): {data_hash.hex()}")

# Function to prevent replay attacks
def prevent_replay_attacks():
    print("\nPreventing replay attacks for devices...")
    previous_timestamps = {}
    previous_nonces = {}

    for device in devices:
        timestamp = int(time.time())
        nonce = os.urandom(16).hex()

        # Check for replay attack
        if device in previous_timestamps and timestamp <= previous_timestamps[device]:
            print(f"Replay attack detected for {device}!")
        else:
            previous_timestamps[device] = timestamp

        if device in previous_nonces and nonce == previous_nonces[device]:
            print(f"Nonce reuse detected for {device}!")
        else:
            previous_nonces[device] = nonce

        print(f"Device: {device} is secure.")

# Function to view logs
def view_logs():
    print("\nActivity Logs:")
    for device, data in sensor_data.items():
        log_entry = {
            "device": device,
            "data": data.decode(),
            "timestamp": int(time.time()),
            "status": "Secure"
        }
        print(log_entry)

# Main menu
def main_menu():
    while True:
        print("\n===== IoT Security System =====")
        print("1. Issue Certificates")
        print("2. Add a New Device")
        print("3. Input Sensor Data")
        print("4. Secure Communication")
        print("5. Ensure Data Integrity")
        print("6. Prevent Replay Attacks")
        print("7. View Logs")
        print("8. Exit")

        choice = input("Enter your choice: ")
        if choice == "1":
            issue_certificate(devices)
        elif choice == "2":
            add_device()
        elif choice == "3":
            input_sensor_data()
        elif choice == "4":
            secure_communication()
        elif choice == "5":
            ensure_data_integrity()
        elif choice == "6":
            prevent_replay_attacks()
        elif choice == "7":
            view_logs()
        elif choice == "8":
            print("Exiting program.")
            break
        else:
            print("Invalid choice! Please try again.")

# Run the program
main_menu()



===== IoT Security System =====
1. Issue Certificates
2. Add a New Device
3. Input Sensor Data
4. Secure Communication
5. Ensure Data Integrity
6. Prevent Replay Attacks
7. View Logs
8. Exit
Enter your choice: 1

Simulating certificate issuance for devices...
Generating certificate for temperature-sensor.smarthome.com...
Certificate for temperature-sensor.smarthome.com generated and signed by Root CA.
Generating certificate for light-sensor.smarthome.com...
Certificate for light-sensor.smarthome.com generated and signed by Root CA.
Generating certificate for cleaning-sensor.smarthome.com...
Certificate for cleaning-sensor.smarthome.com generated and signed by Root CA.

===== IoT Security System =====
1. Issue Certificates
2. Add a New Device
3. Input Sensor Data
4. Secure Communication
5. Ensure Data Integrity
6. Prevent Replay Attacks
7. View Logs
8. Exit
Enter your choice: 2
Enter the new device name (e.g., 'motion-sensor.smarthome.com'): temperature-sensor.smarthome.com
Device alre